In [16]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess

import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def preprocess(text):
    tokens = simple_preprocess(text,deacc= True)
    return [word for word in tokens if word not in stop_words]

In [122]:
# Correct full import block
from qdrant_client.http.models import PointStruct
# from qdrant_client.http.models import PointBatch

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams,PointStruct

## Docker Command - docker start qdrant

class Qdrant():

    def __init__(self,host = "localhost", port=6333):
        try:
            self.client=QdrantClient(host=host,port=port)
        except Exception as e:
            raise ConnectionError(f"Qdrant Connection failed {e}")

    
    def create_collection(self,collection_name,vector_size):
        try:
            self.client.recreate_collection(
            collection_name = collection_name,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),)
        except Exception as e:
            return f"Error Creating Colletion: {e}"


    def add_vectors(self,collection_name,id,vector):

        try:
            self.client.upsert(
            collection_name=collection_name,
            points=[PointStruct(id = id,vector=vector)])
        except Exception as e:
            return f"Error adding vectors: {e}"

    def add_bulk_vectors(self,collection_name,points):
        try:
            self.client.upload_collection(collection_name=collection_name,
                                      points=points)

        except Exception as e:
            return f"Error retreiving data: {e}"

    def retreve(self,collection_name,ids,with_vectors= True):
        try:
            data = self.client.retrieve(
                collection_name=collection_name,
                ids=ids,
                with_vectors= with_vectors)
            return data
        except Exception as e:
            return f"Error retreiving data: {e}"

    
    def search(self,collection_name,query_vector,limit):
        try:
            result = self.client.search(collection_name = collection_name,
                                    query_vector=query_vector,limit = limit )
            return result
        except Exception as e:
            return f"Error searching vectors: {e}"
        
    def check_collections(self):
        try:
            result = self.client.get_collections()
            return result
        except Exception as e:
            return f"Error fetching collections: {e}"
        
    def delete_collect(self,collection_name):
        try:
            self.client.delete_collection(collection_name=collection_name)
        
        except Exception as e:
            return f"Error Deleting Collection: {e}"
        
    def count_vecs(self):
        collection_info  =self.client.get_collection("products")
        return collection_info.vectors_count




# # collection_info = client.get_collection("your_collection_name")
# print(f"Number of vectors: {collection_info.vectors_count}")



# print(qd.count_vecs())



# hits = client.search(collection_name = "test_collection",query_vector =search_vec,limit = 4 )

# # Step 2: Upload vectors
# client.upsert(
#     collection_name="test_collection",
#     points=[
#         PointStruct(id=i+1, vector= vec) for i , vec in enumerate(demo_vecs)
         
#     ]
# )


# result = client.retrieve(collection_name="test_collection", ids = [10],with_vectors=True)




In [7]:
import json
data = []
with open(r"C:\Users\Rishabh\Desktop\Python New\Amazon Recommender System\product_data\Appliances\meta_Appliances.jsonl",'r',encoding = "utf-8") as file:
    for line in file:
        data_line = json.loads(line.strip())
        data.append(data_line)
        

In [11]:
df =pd.DataFrame(data)

In [14]:
df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"ROVSUN Ice Maker Machine Countertop, Make 44lb...",3.7,61,[【Quick Ice Making】This countertop ice machine...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Our Point of View on the Euhomy Ic...,ROVSUN,"[Appliances, Refrigerators, Freezers & Ice Mak...","{'Brand': 'ROVSUN', 'Model Name': 'ICM-2005', ...",B08Z743RRD,None,NaN,NaN
1,Tools & Home Improvement,"HANSGO Egg Holder for Refrigerator, Deviled Eg...",4.2,75,"[Plastic, Practical Kitchen Storage - Our egg ...",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': '10 Eggs Egg Holder for Refrigerato...,HANSGO,"[Appliances, Parts & Accessories, Refrigerator...","{'Manufacturer': 'HANSGO', 'Part Number': 'HAN...",B097BQDGHJ,None,NaN,NaN
2,Tools & Home Improvement,"Clothes Dryer Drum Slide, General Electric, Ho...",3.5,18,[],"[Brand new dryer drum slide, replaces General ...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],GE,"[Appliances, Parts & Accessories]","{'Manufacturer': 'RPI', 'Part Number': 'WE1M33...",B00IN9AGAE,None,NaN,NaN
3,Tools & Home Improvement,154567702 Dishwasher Lower Wash Arm Assembly f...,4.5,26,[MODEL NUMBER:154567702 Dishwasher Lower Wash ...,[MODEL NUMBER:154567702 Dishwasher Lower Wash ...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],folosem,"[Appliances, Parts & Accessories, Dryer Parts ...","{'Manufacturer': 'folosem', 'Part Number': '15...",B0C7K98JZS,None,NaN,NaN
4,Tools & Home Improvement,Whirlpool W10918546 Igniter,3.8,12,[This is a Genuine OEM Replacement Part.],[Whirlpool Igniter],25.07,[{'thumb': 'https://m.media-amazon.com/images/...,[],Whirlpool,"[Appliances, Parts & Accessories]","{'Manufacturer': 'Whirlpool', 'Part Number': '...",B07QZHQTVJ,None,NaN,NaN


In [19]:

df['cleaned_text'] = df['title'].apply(lambda x: preprocess(str(x) if pd.notnull(x) else ""))


In [20]:
model = gensim.models.Word2Vec(df['cleaned_text'],min_count = 1)


In [24]:


def tokens_to_vec(token_text, vector_size=100):
    # Filter tokens in the model's vocabulary
    valid_tokens = [token for token in token_text if token in model.wv]
    
    # Handle empty valid_tokens gracefully
    if not valid_tokens:
        return np.zeros((1, vector_size))  # Return zero vector if no valid tokens
    
    # Get embeddings for valid tokens
    token_embeddings = model.wv[valid_tokens]
    
    # Compute mean embedding
    mean_embedding = np.mean(token_embeddings, axis=0)

    return mean_embedding
    

In [26]:
import numpy as np
df['vecs'] = df['cleaned_text'].apply(lambda x: list(tokens_to_vec(x)))   

In [75]:
df.columns

Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together', 'subtitle', 'author',
       'cleaned_text', 'vecs'],
      dtype='object')

In [132]:
product_id  =1
vec = df['vecs'].iloc[0]

product_id_2  = 2
vec_2 = df['vecs'].iloc[10]

product_id_3  = 3
vec_3 = df['vecs'].iloc[10]


In [136]:
all_vecs = [{'id':product_id,'vec':vec }, {'id':product_id_2,'vec':vec_2 },{'id':product_id_3,'vec':vec_3 }]

In [143]:
# all_vecs[1]['vec']

In [130]:
# demo_dict=[PointStruct(id= product_id, vector = vec)]

In [131]:
def chunkify(data,batch_size):
    for i in range(0,len(data),batch_size):
        batch = data[i:i+batch_size]
        yield batch

In [152]:
chunkify(all_vecs,1)

<generator object chunkify at 0x000001AA03A4FF10>

In [154]:
for index, a in enumerate(chunkify(all_vecs,1)):
    qd.add_vectors(collection_name='product_date',id =all_vecs[index]['id'],vector = all_vecs[index]['vec'])
    print(index)


0
1
2


In [145]:
qd = Qdrant()
for i in range(len(all_vecs)):
    qd.add_vectors(collection_name='product_date',id =all_vecs[i]['id'],vector = all_vecs[i]['vec'])
    

In [158]:
# qd.retreve(collection_name='product_date',ids =[3])

In [46]:
# qd.create_collection(collection_name ='product_date',vector_size = 100)

In [159]:
vec_points = [
    PointStruct(id=row['title'], vector=row['vecs'])
    for row in df.to_dict('records')
]

In [167]:
# vec_points[1]

In [73]:

# vec_points = [
#     PointStruct(id=row['title'], vector=row['vecs'])
#     for row in df.to_dict('records')
# ]
